In [92]:
# Set up for running the code
import pandas
import numpy
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from yellowbrick.classifier import ClassificationReport,ConfusionMatrix


# Read in the data from the CSV file
dataframe = pandas.read_csv("Crime_Data_from_2020_to_Present.csv", dtype={'TIME OCC': str})

In [93]:
# Get only the date in the DATE OCC column
dataframe['DATE OCC'] = dataframe['DATE OCC'].str.split().str.get(0)

In [95]:
# Change the type of the column to date
dataframe['DATE OCC'] = pandas.to_datetime(dataframe['DATE OCC'], format='%m/%d/%Y') 

In [103]:
dataframe['DATE OCC'] = dataframe['DATE OCC'].numpy.dtypes.DateTime64DType

AttributeError: 'Series' object has no attribute 'numpy'

In [96]:
# Change the TIME OCC column to time
dataframe['hour'] = pandas.to_datetime(dataframe['TIME OCC'], format='%H%M').dt.hour
dataframe['minute'] = pandas.to_datetime(dataframe['TIME OCC'], format='%H%M').dt.minute

In [104]:
# get the sample
trainingSet = dataframe.sample(frac=0.8)
testingSet = dataframe[~dataframe.index.isin(trainingSet.index)]


# XTrain = trainingSet[['AREA', 'DATE OCC', 'hour', 'minute']]
XTrain = trainingSet[['AREA', 'hour', 'minute']]
yTrain = trainingSet['Crm Cd']

# xTest = testingSet[['AREA', 'DATE OCC', 'hour', 'minute']]
xTest = testingSet[['AREA', 'hour', 'minute']]
yTest = testingSet['Crm Cd']



In [105]:
# Create the model

rfcModel = ensemble.RandomForestClassifier()
rfcModel.fit(XTrain, yTrain)
# dataframe['DATE OCC']

RandomForestClassifier()

In [107]:
rfcTestPrediction = rfcModel.predict(xTest)



In [108]:
RFCAccuracy = metrics.accuracy_score(yTest, rfcTestPrediction)
print(RFCAccuracy)

0.15009558443499324
